"""
import numpy as np
import pandas as pd
from flask import Flask, request, jsonify, render_template
import pickle

from flask import Response
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
import json, random
import matplotlib.pyplot as plt
import io
import base64

# Create flask app
flask_app = Flask(__name__, template_folder='temp')


model = pickle.load(open("HMM_1.pkl", "rb"))

@flask_app.route('/')
def home():
    return render_template('index.html')

@flask_app.route("/predict", methods = ["POST", "GET"])
def predict():
    """
    cur = mysql.connection.cursor()
    cur.execute()
    """  
    df = pd.read_csv("/Volumes/Projects/DAZOQ-internship/one_sensor.csv")
    
    df_data = df[['timestamp_utc','total_system_power']]
    # Features and Labels
    df_x = df_data['total_system_power'].values


    prediction = model.predict(df_x.reshape(-1, 1))
    
    df_data['cluster'] = prediction
    df_data.to_csv("/Volumes/Projects/DAZOQ-internship/Cluster.csv", index=False)
    return render_template('index.html', prediction_text = "group point{}".format(prediction))

if __name__ == "__main__":
    flask_app.run(debug=True, use_reloader=False)
"""

In [19]:
#, use_reloader=False.........."group point{}".format(prediction)======render_template('index.html', prediction_text = "group point{}".format(prediction))





In [2]:
!pip3 install flask

  Obtaining dependency information for flask from https://files.pythonhosted.org/packages/93/a6/aa98bfe0eb9b8b15d36cdfd03c8ca86a03968a87f27ce224fb4f766acb23/flask-3.0.2-py3-none-any.whl.metadata
  Obtaining dependency information for itsdangerous>=2.1.2 from https://files.pythonhosted.org/packages/68/5f/447e04e828f47465eeab35b5d408b7ebaaaee207f48b7136c5a7267a30ae/itsdangerous-2.1.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.3/101.3 kB 3.1 MB/s eta 0:00:00


# Visulaization 

# visualization with input

In [ ]:
import numpy as np
import pandas as pd
from flask import Flask, request, jsonify, render_template
import pickle
import datetime as dt

from flask import Response
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
import json, random
import matplotlib.pyplot as plt
import io
import base64
from PIL import Image
import urllib.parse


prediction = []

# Create flask app
flask_app = Flask(__name__, template_folder='temp')

model = pickle.load(open("HMM_2.pkl", "rb"))

@flask_app.route('/')
def home():
    return render_template('index1.html')

@flask_app.route("/predict", methods = ["POST", "GET"])
def predict():
    """
    cur = mysql.connection.cursor()
    cur.execute()
    """  

    
    # Get values through input bars
    start_date = request.form["date_s"] 
    end_date = request.form["date_e"]
    c_id = int(request.form["Company_id"])
    p_id = int(request.form["Process_id"])
    sp_id = int(request.form["Subprocess_id"])
    e_id = int(request.form["Equipment_id"])
    
    
    
    df = pd.read_csv("Data/sensor_one_week1001.csv")
    #convert the timestamp 
    
    df['timestamp_utc'] = pd.to_datetime(df['timestamp_utc'])
    df['timestamp_utc']
    
    
    df1 = df.loc[(df['timestamp_utc'].between(start_date, end_date)) & (df['company_id']== c_id) & (df['process_id']== p_id) & (df['sub_process_id']== sp_id) & (df['equipment_id']== e_id)] 
    #print(df1)
    df1 = df1.sort_values(by = 'timestamp_utc')
    df1.to_csv("Data/df_mask.csv", index=False)
    df_data = df1[['timestamp_utc','total_system_power']]
    #df_x = df_data['total_system_power'].values
    df_data['smooth'] = df_data['total_system_power'].rolling(window=20, win_type='gaussian', center=True).mean(std=2)
    df_data = df_data.dropna().copy()
    df_x = df_data['smooth'].values
    prediction = model.predict(df_x.reshape(-1, 1))
    df_data['cluster'] = prediction
    df_data.to_csv("Data/Cluster1.csv", index=False)
    
    
    df_display = pd.read_csv("Data/Cluster1.csv")
    df_display['timestamp_utc'] = pd.to_datetime(df_display['timestamp_utc'])
    data = {
        'a': df_display['timestamp_utc'].dt.strftime('%d,%H:%M'),
        'b': df_display['smooth'],
        'c': df_display['cluster']
    }
    
    from datetime import datetime
#plt.xticks(np.arange(0, len(a) + 1, 5))
#plt.plot(figsize=(20, 10))
    plt.rcParams.update({'figure.figsize':(20,10), 'figure.dpi':300})
    plt.xticks(np.arange(0, len(df_display['timestamp_utc']) + 1, 20), rotation = 60)
    #plt.scatter('a','b ',c=, cmap= 'rainbow')
    
    im = plt.scatter(df_display['timestamp_utc'].dt.strftime('%d,%H:%M'), 'b', c='c', data=data, cmap = 'rainbow')
    #plt.xlabel('X label')
    plt.ylabel('Y label')
   
    #plt.savefig(os.path.join('plot.png'))

    #return render_template('index.html')

    data = io.BytesIO()
    plt.savefig(data, format = 'png')
    data.seek(0)
    encoded_img_data = urllib.parse.quote(base64.b64encode(data.read()).decode())
    #mime = "image/jpeg"
    
    #file = "data:%s;base64,%s"%(mime, img_base64)

    return render_template("index1.html", image=encoded_img_data)



if __name__ == "__main__":
    flask_app.run(debug=True, use_reloader=False)



 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [06/Mar/2024 08:57:05] "GET / HTTP/1.1" 200 -
/var/folders/y9/kzh00ysx0cl0fd_lkcz27c_00000gn/T/ipykernel_54623/4067236820.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data['smooth'] = df_data['total_system_power'].rolling(window=20, win_type='gaussian', center=True).mean(std=2)
127.0.0.1 - - [06/Mar/2024 08:57:36] "POST /predict HTTP/1.1" 200 -


In [ ]:
 """
    if c_id in df.company_id.values:
        if p_id in df.process_id.values:
            if sp_id in df.subprocess_id.values:
                if e_id in df.equipment_id.values:
                    
                    df_data = df[['timestamp_utc','total_system_power']]
                    df_x = df_data['total_system_power'].values
                    prediction = model.predict(df_x.reshape(-1, 1))
    
                    df_data['cluster'] = prediction
                    df_data.to_csv("/Volumes/Projects/DAZOQ-internship/Cluster1.csv", index=False)
                    
    else:
        print('Values not found')
""" 

In [1]:
import numpy as np
import pandas as pd
from flask import Flask, request, jsonify, render_template
import pickle
import datetime as dt

from flask import Response
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
import json, random
import matplotlib.pyplot as plt
import io
import base64
from PIL import Image
import urllib.parse

In [2]:
model = pickle.load(open("HMM_3.pkl", "rb"))

In [3]:
df = pd.read_csv("/Volumes/Projects/DAZOQ-internship/sensor_one_week1001.csv")
#convert the timestamp 

df['timestamp_utc'] = pd.to_datetime(df['timestamp_utc'])
df['timestamp_utc']


0       2023-03-13 05:04:00+00:00
1       2023-03-13 05:04:00+00:00
2       2023-03-13 05:04:00+00:00
3       2023-03-13 05:04:00+00:00
4       2023-03-13 05:04:00+00:00
                   ...           
42485   2023-03-13 05:02:00+00:00
42486   2023-03-13 05:02:00+00:00
42487   2023-03-13 05:02:00+00:00
42488   2023-03-13 05:02:00+00:00
42489   2023-03-13 05:02:00+00:00
Name: timestamp_utc, Length: 42490, dtype: datetime64[ns, UTC]

In [10]:
df1 = df.loc[(df['timestamp_utc'].between('2023-03-13 05:04:00+00:00', '2023-03-13 05:20:00+00:00')) & (df['company_id']== 1001) & (df['process_id']== 3) & (df['sub_process_id']== 1) & (df['equipment_id']== 1)]

In [11]:
df_data = df1[['timestamp_utc','total_system_power']]
df_x = df_data['total_system_power'].values
print(df_data)
prediction = model.predict(df_x.reshape(-1, 1))
df_data['cluster'] = prediction
df_data.to_csv("/Volumes/Projects/DAZOQ-internship/Cluster_example.csv", index=False)


                timestamp_utc  total_system_power
20  2023-03-13 05:04:00+00:00           10.889110
23  2023-03-13 05:06:00+00:00            9.589526
41  2023-03-13 05:08:00+00:00            9.912909
54  2023-03-13 05:10:00+00:00            9.920809
65  2023-03-13 05:12:00+00:00           11.227239
86  2023-03-13 05:14:00+00:00           11.197217
103 2023-03-13 05:16:00+00:00            5.127493
124 2023-03-13 05:18:00+00:00            5.121963
144 2023-03-13 05:20:00+00:00            5.079039


<ipython-input-11-eeedb5e62f0a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data['cluster'] = prediction


In [12]:
df_data

,timestamp_utc,total_system_power,cluster
20,2023-03-13 05:04:00+00:00,10.889110,2
23,2023-03-13 05:06:00+00:00,9.589526,3
41,2023-03-13 05:08:00+00:00,9.912909,3
54,2023-03-13 05:10:00+00:00,9.920809,3
65,2023-03-13 05:12:00+00:00,11.227239,3
86,2023-03-13 05:14:00+00:00,11.197217,3
103,2023-03-13 05:16:00+00:00,5.127493,3
124,2023-03-13 05:18:00+00:00,5.121963,3
144,2023-03-13 05:20:00+00:00,5.079039,3
